In [ ]:
!pip install openpyxl

In [ ]:
import json
import re
import os
from openpyxl import Workbook

def extract_urls_from_file(filename):
    with open(filename, 'r', encoding='cp1251') as file:
        lines = file.readlines()

    results = []
    i = 0

    while i < len(lines):
        line = lines[i].strip()
        if not line:
            i += 1
            continue

        # Проверяем, является ли строка UUID (id)
        if re.match(r'^[a-f0-9]{8}-[a-f0-9]{4}-[a-f0-9]{4}-[a-f0-9]{4}-[a-f0-9]{12}$', line, re.I):
            current_id = line
            i += 1

            # Следующая строка должна быть JSON
            if i < len(lines):
                json_line = lines[i].strip()
                try:
                    data = json.loads(json_line)
                    urls = data.get("urls", {})
                    id_urls = urls.get(current_id, {})
                    live_data = id_urls.get("live", {})
                    url_list = live_data.get("url", [])

                    for url in url_list:
                        results.append((current_id, url))
                except json.JSONDecodeError as e:
                    print(f"Ошибка парсинга JSON для ID {current_id}: {e}")
            else:
                print(f"Отсутствует JSON для ID {current_id}")
        else:
            i += 1

        i += 1

    return results

# Основной код
if __name__ == "__main__":
    # Пути
    input_file = "/content/drive/MyDrive/Г/curls/names.txt"
    output_folder = "/content/drive/MyDrive/Г/curls/"
    output_file = os.path.join(output_folder, "results.xlsx")

    # Извлечение данных
    print("Чтение и обработка файла...")
    extracted_data = extract_urls_from_file(input_file)

    # Запись в Excel
    print(f"Сохранение результатов в {output_file}...")
    wb = Workbook()
    ws = wb.active
    ws.title = "URLs"
    ws.append(["ID", "URL"])

    for uid, url in extracted_data:
        ws.append([uid, url])

    wb.save(output_file)
    print(f"✅ Готово! Результат сохранён: {output_file}")

Чтение и обработка файла...
Сохранение результатов в /content/drive/MyDrive/Г/curls/results.xlsx...
✅ Готово! Результат сохранён: /content/drive/MyDrive/Г/curls/results.xlsx
